In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('mylogregcodealong').getOrCreate()

24/04/06 12:49:31 WARN Utils: Your hostname, Luiss-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.100.206 instead (on interface en1)
24/04/06 12:49:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/06 12:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/06 12:49:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/06 12:49:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/06 12:49:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

In [9]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [10]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [11]:
embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder= OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [12]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'], outputCol='features')

In [13]:
from pyspark.ml.classification import LogisticRegression


In [14]:
from pyspark.ml import Pipeline

In [16]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [17]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic])

In [18]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [19]:
fit_model = pipeline.fit(train_data)

24/04/06 13:05:43 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [20]:
results = fit_model.transform(test_data)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [25]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [26]:
AUC = my_eval.evaluate(results)

In [27]:
AUC

0.7560606060606061